In [1]:
import pickle
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import read_csv
import datetime as dt
from matplotlib import pyplot
import matplotlib.pyplot as plt

In [2]:
filepath= "/Users/yanjia/Documents/extubation/processed_data/processedFrames.pkl"
OutputFrames = pickle.load(open(filepath, 'rb'))

In [3]:
df= DataFrame()
for key, chartFrame in OutputFrames.items():
    chartFrame['extubation_failure'] = np.nan    
    for i in chartFrame.index:
        if i == 0:
            continue        
        if chartFrame.loc[i, 'Vented'] == 0 and chartFrame.loc[i-1, 'Vented'] == 1:
            for j in range(48):
                #print(j)
                if i+j in chartFrame.index:   
                    if chartFrame.loc[i+j,'Vented'] == 1:
                        chartFrame.loc[i-1, 'extubation_failure'] = 1
                        break
                    else:
                        chartFrame.loc[i-1, 'extubation_failure'] = 0
            df = df.append(chartFrame.iloc[i-1], ignore_index=True)

In [8]:
cohort = df[df['extubation_failure'] ==1]

In [9]:
len(cohort['hadm'].unique())

434

In [4]:
ventilator = df['Ventilator Mode'].unique()

In [5]:
ventilator.shape

(16,)

In [6]:
np.sort(ventilator)

array([ 0.,  1.,  2.,  7., 10., 11., 12., 13., 17., 26., 30., 47., 48.,
       49., 51., 53.])

In [7]:
ventilator_dict = {0.: 0,  1.:1,  2.:2,  7.:3,  10.:4, 11.:5, 12.:6, 13.:7, 17.:8,
       26.:9, 30.:10, 47.:11, 48.:12, 49.:13, 51.:14, 53.:15}

In [8]:
convert_dict = {'Admittype': int, 
                'Ethnicity': int,
                'Gender': int,
                'SBT_value': int,
                'O2 saturation pulseoxymetry': int,
                'Peak Insp. Pressure': int
                
               }

In [9]:
df['ventilator_category'] = df['Ventilator Mode'].apply(lambda x:ventilator_dict[x] )

In [10]:
df = df.rename(columns={'Respiratory Rate (spontaneous)': 'Respiratory Rate_spontaneous'})

In [11]:
df = df.drop(['timestamp','firstICU', 'subject', 'Admdays', 'Height (cm)', 'Vented', 'SBT', 'Ventilator Mode', 'Non Invasive Blood Pressure mean', 'Non Invasive Blood Pressure systolic', 'Non Invasive Blood Pressure diastolic', 'Propofol','Fentanyl (Concentrate)', 'Midazolam (Versed)','Fentanyl','Dexmedetomidine (Precedex)','Morphine Sulfate','Hydromorphone (Dilaudid)','Lorazepam (Ativan)'], axis=1)

In [12]:
df.shape

(6696, 27)

In [13]:
df.columns

Index(['Admission Weight (Kg)', 'Admittype', 'Age',
       'Arterial Blood Pressure diastolic', 'Arterial Blood Pressure mean',
       'Arterial Blood Pressure systolic', 'Arterial CO2 Pressure',
       'Arterial O2 pressure', 'Ethnicity', 'Gender', 'Heart Rate',
       'Inspired O2 Fraction', 'Mean Airway Pressure', 'O2 Flow',
       'O2 saturation pulseoxymetry', 'PEEP set', 'PH (Arterial)',
       'Peak Insp. Pressure', 'Plateau Pressure', 'Respiratory Rate',
       'Respiratory Rate_spontaneous', 'Richmond-RAS Scale', 'SBT_value',
       'Tidal Volume (observed)', 'extubation_failure', 'hadm',
       'ventilator_category'],
      dtype='object')

In [14]:
df1 = df.dropna()

In [15]:
df1.shape

(2841, 27)

In [16]:
df1 = df1.astype(convert_dict) 

In [17]:
one = df1[df1['SBT_value'] == -1]

In [18]:
one.shape

(72, 27)

In [19]:
df1.to_csv('./data/orginal_data.csv', index= False)

In [20]:
######### df2 is the dataset that delete sbt failed patients when they are extubated ###########
df2 = DataFrame()
for i in df1.index:
    if df1.loc[i, 'SBT_value'] != -1:
        df2 = df2.append(df1.loc[i], ignore_index=True)

In [21]:
df2.shape

(2769, 27)

In [22]:
df2.to_csv('./data/orginal_data_non_sbtfail.csv', index= False)

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
test_size = 0.2
test_split_random_state = 1

In [25]:
df1 = df1.drop(['hadm', 'SBT_value'], axis=1)

In [26]:
df2 = df2.drop(['hadm', 'SBT_value'], axis=1)

In [27]:
def split_data(data):
    train_df, test_df = train_test_split(
            data, test_size=test_size, random_state=test_split_random_state)
    return train_df, test_df

In [28]:
train_df, test_df = split_data(df1)

In [29]:
train_no_sbtfail, test_no_sbtfail = split_data(df2)

In [30]:
train_df.to_csv('./data/train_data.csv', index= False)
test_df.to_csv('./data/test_data.csv', index= False)

In [31]:
train_no_sbtfail.to_csv('./data/train_data1.csv', index= False)
test_no_sbtfail.to_csv('./data/test_data1.csv', index= False)

In [32]:
one_train = train_df[train_df['extubation_failure'] == 1]

In [33]:
zero_train= train_df[train_df['extubation_failure'] == 0]

In [34]:
one_train.shape, zero_train.shape, train_df.shape

((230, 25), (2042, 25), (2272, 25))

In [35]:
one_nosbt = train_no_sbtfail[train_no_sbtfail['extubation_failure'] == 1]

In [36]:
one_nosbt.shape, train_no_sbtfail.shape

((218, 25), (2215, 25))

In [37]:
one = df2[df2['extubation_failure'] ==1]

In [38]:
df2.shape, one.shape

((2769, 25), (275, 25))